In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
import transformers
from transformers import BertForSequenceClassification
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()
client

In [5]:


response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "You are a speech act classifier that uses its expert knowledge of linguistics and speech act theory to make speech act classifications for a sequence of utterances. Utterances may be classified as one of the four following speech acts: Commisive, Directive, Informative, and Question.\n\n1. Commissive\nA commissive speech act is a type of speech act in linguistic pragmatics where the speaker commits to a future course of action or a particular state of affairs. In other words, it involves the speaker making a promise, commitment, or pledge to do something in the future. Commissive speech acts are performative in nature, meaning that the utterance of the words itself brings about the commitment or promise.\n\nKey features of commissive speech acts include:\n\nVerbs of Commitment or Promise: Commissive speech acts often involve the use of specific verbs that indicate commitment or promise, such as \"promise,\" \"commit,\" \"swear,\" \"guarantee,\" or \"pledge.\"\n\nFuture-Oriented Language: The speech act typically includes language expressing a commitment or intention to perform a future action. Phrases using future tenses like \"I will,\" \"I shall,\" or \"I am going to\" are common.\n\nFirst-Person Pronouns: Commissive speech acts are often expressed in the first person, using pronouns like \"I\" or \"we,\" as the speaker is personally committing to an action or promise.\n\nExplicit Declarations: The commitment or promise is usually explicitly stated, making the speech act clear and direct. The speaker may make an explicit declaration of their intention.\n\nPerformative Nature: Uttering the words of a commissive speech act is performative, meaning that saying the words brings about the commitment itself. For example, saying \"I promise\" is an act of making a promise.\n\nAssurance and Certainty: The speaker may use words conveying assurance or certainty to emphasize the commitment. Terms like \"assure,\" \"guarantee,\" or \"undertake\" may be present.\n\nExamples of commissive speech acts include statements like \"I promise to help you,\" \"I am committed to finishing the project on time,\" or \"I guarantee that I will be there.\" These speech acts play a significant role in interpersonal communication by expressing the speaker's commitment to future actions or obligations.\n\n2. Directive\nA directive speech act is a type of speech act in linguistic pragmatics where the speaker attempts to get the listener to do something. It involves the speaker influencing the behavior of the listener by making a request, giving a command, or offering an invitation. Directive speech acts are communicative acts through which the speaker aims to bring about a specific action or response from the listener.\n\nKey characteristics of directive speech acts include:\n\nImperative Verbs: These are verbs that convey commands or requests, such as \"do,\" \"please,\" \"stop,\" \"give,\" etc. Imperative sentences are often used to express directive speech acts.\n\nRequests: The speaker may use polite or direct language to ask the listener to perform a certain action. Phrases like \"Could you,\" \"Would you,\" or \"Please\" are common in directive speech acts.\n\nCommanding Language: Direct and explicit language is often used to issue commands. Imperative sentences, which lack a subject and begin with a verb, are a typical form of commanding language.\n\nPoliteness Strategies: Depending on cultural and social norms, speakers may employ politeness strategies when making requests or giving directives. This can include using softening expressions or framing requests in a more polite manner.\n\nQuestions as Requests: Sometimes, questions can be used as a subtle form of requesting action. For example, \"Can you pass the salt?\" is a request disguised as a question.\n\nContextual Clues: Understanding the context of the conversation is crucial for correctly interpreting directive speech acts. The same words or phrases can have different implications based on the context in which they are used.\n\nExamples of directive speech acts include requests like \"Please close the door,\" commands like \"Stop talking,\" and invitations like \"Join us for dinner.\" The effectiveness of a directive speech act depends on the listener's willingness to comply and the social dynamics between the speaker and the listener.\n\n\n3. Informative:\nAn informative speech act is a type of speech act in linguistic pragmatics where the speaker provides information or conveys knowledge to the listener. The primary goal of an informative speech act is to share facts, describe events, explain concepts, or communicate any form of information to the audience. Unlike other speech acts that may involve making promises, requests, or commands, informative speech acts focus on the transmission of information.\n\nKey characteristics of informative speech acts include:\n\nDeclarative Statements: Informative speech acts often take the form of declarative statements, where the speaker asserts a fact or provides information without necessarily eliciting a specific response from the listener.\n\nDescriptive Language: The language used in informative speech acts is typically descriptive, aiming to provide a clear and accurate portrayal of a subject, event, or concept.\n\nObjective Tone: The speaker adopts an objective and neutral tone when presenting information. The goal is to convey facts without expressing personal opinions or emotions.\n\nExplanations and Clarifications: Informative speech acts may involve explanations or clarifications to ensure that the information is understood by the audience. This could include defining terms, offering examples, or providing additional context.\n\nUse of Evidence: Speakers may support their informative speech acts by presenting evidence, data, or examples to enhance the credibility and reliability of the information being shared.\n\nEducational Intent: Informative speech acts are often employed in educational or instructional contexts where the speaker aims to enhance the listener's knowledge on a particular subject.\n\nExamples of informative speech acts include:\n\n\"The capital of France is Paris.\"\n\"Water boils at 100 degrees Celsius at sea level.\"\n\"The Industrial Revolution began in the late 18th century.\"\nIn everyday communication, informative speech acts are prevalent in educational settings, presentations, news reporting, and any situation where the primary purpose is to convey information rather than to perform an action or express emotions.\n\n4. Question\nA question speech act is a type of speech act in linguistic pragmatics where the speaker seeks information, clarification, or confirmation from the listener. The primary purpose of a question is to elicit a response or a specific piece of information from the person being addressed. Questions can take various forms and serve different communicative functions.\n\nKey characteristics of question speech acts include:\n\nInterrogative Form: Questions are typically formed using interrogative words, phrases, or structures. These can include question words like \"who,\" \"what,\" \"where,\" \"when,\" \"why,\" and \"how,\" as well as inversion of subject and verb.\n\nSeeking Information: The main goal of a question is to obtain information that the speaker does not currently possess. This information could relate to facts, opinions, experiences, or any other relevant details.\n\nOpen-Ended or Closed-Ended: Questions can be classified as open-ended or closed-ended. Open-ended questions allow for a more detailed and expansive response, while closed-ended questions usually have a limited set of possible answers and often require a simple \"yes\" or \"no.\"\n\nRhetorical Questions: In some cases, speakers may pose rhetorical questions, which are not meant to elicit a direct answer but rather to emphasize a point, express a statement, or engage the listener's thought.\n\nIntonation and Tone: The intonation and tone used when asking a question can convey additional information about the speaker's intent. For example, rising intonation at the end of a statement can sometimes indicate a question.\n\nContextual Relevance: The interpretation of a question often depends on the broader context of the conversation. Understanding the context helps in providing appropriate and relevant answers.\n\nExamples of question speech acts include:\n\n\"What is your name?\"\n\"Where did you go last night?\"\n\"How does this machine work?\"\n\"Did you enjoy the movie?\"\nQuestions play a crucial role in communication by facilitating the exchange of information, promoting dialogue, and engaging participants in conversation. They can be used in various social situations, such as interviews, casual conversations, educational settings, and problem-solving interactions.\n\nUse the following steps to make speech act classifications for utterances:\n\nStep 1: Identify the key linguistic and semantic features in the utterance.\nStep 2: Compare the key linguistic and semantic features of the utterance to the features of the dialogue acts.\nStep 3: Use this comparison to predict the dialogue act class that the utterance most likely belongs to"
    },
    {
      "role": "user",
      "content": "I will give you a sequence of utterances in the form:\n\n'''\n1. first utterance\n2. second utterance\n3. third utterance\nn. nth utterance\n'''\n\nPlease return the most accurate classification tag for all n utterances in the sequence in the following list format:\n\n'''\nClassifications:\n[classification for 1st utterance, classification for 2nd utterance, classification for 3rd utterance, classification for nth utterance]\n'''\n\nAdditionally, please return an explanation of your classification process for each utterance in the following format:\n\n'''\nExplanations:\n1. explanation for first classification\n2. explanation for second classification\n3. explanation for third classification\nn. explanation for nth classification\n'''"
    },
    {
      "role": "user",
      "content": "6. that's a good idea . i hear mary and sally often go there to play pingpong.perhaps we can make a foursome with them .\n7. sounds great to me ! if they are willing , we could ask them to go dancing with us.that is excellent exercise and fun , too .\n8. good.let ' s go now .\t\n9. all right, i will do that ."
    }
  ],
  temperature=0.32,
  max_tokens=704,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [6]:
response

ChatCompletion(id='chatcmpl-8h8tnUnmHfGYom63NyVIwnHFsBQfc', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Classifications:\n[Informative, Informative, Directive, Commissive]\n\nExplanations:\n1. The first utterance "that\'s a good idea. I hear Mary and Sally often go there to play pingpong. Perhaps we can make a foursome with them." is classified as Informative. It provides information about Mary and Sally going to play pingpong and suggests the possibility of joining them.\n2. The second utterance "Sounds great to me! If they are willing, we could ask them to go dancing with us. That is excellent exercise and fun, too." is also classified as Informative. It expresses agreement and suggests asking Mary and Sally to go dancing, providing information about the benefits of dancing.\n3. The third utterance "Good. Let\'s go now." is classified as Directive. It contains a command to go now, indicating a specific action to be taken.\n4. The fourth utter

In [3]:
ds = load_dataset("silicone", "dyda_da")

Found cached dataset silicone (/home/dck5549/.cache/huggingface/datasets/silicone/dyda_da/1.0.0/af617406c94e3f78da85f7ea74ebfbd3f297a9665cb54adbae305b03bc4442a5)
100%|██████████| 3/3 [00:00<00:00, 583.33it/s]


In [4]:
ds['train']

Dataset({
    features: ['Utterance', 'Dialogue_Act', 'Dialogue_ID', 'Label', 'Idx'],
    num_rows: 87170
})

In [24]:
l = ds['train'][0:500]["Utterance"]


In [7]:
class BertDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        super(BertDataset, self).__init__()
        self.tokenizer=tokenizer
        self.data = data
        self.max_length=max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        

        utterance = self.data['Utterance'][index]

        inputs = self.tokenizer.encode_plus(
            utterance,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]

        label = np.zeros(4)
        label[self.data["Label"][index]] = 1

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(label, dtype=torch.long)
            }

In [8]:
batch_size = 64

tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased")

train_dataset = BertDataset(ds['train'], tokenizer, max_length=256)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = BertDataset(ds['validation'], tokenizer, max_length=256)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [10]:
for i in train_loader:
    print(i['ids'])
    break

tensor([[  101,  2821,  1010,  ...,     0,     0,     0],
        [  101,  4952,  1012,  ...,     0,     0,     0],
        [  101,  2748,  1010,  ...,     0,     0,     0],
        ...,
        [  101,  2115, 12293,  ...,     0,     0,     0],
        [  101,  2017,  2323,  ...,     0,     0,     0],
        [  101,  1045,  1005,  ...,     0,     0,     0]])
